In [1]:
!pip install catboost
!pip install xgboost
!pip install openml
!pip install numba

In [2]:
#specify GPU to use
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
from sklearn.model_selection import train_test_split
import openml
import category_encoders as ce
import numpy as np
import sklearn

# Load delays_zurich_transport dataset
dataset = openml.datasets.get_dataset(537, download_data=True, download_qualities=True, download_features_meta_data=True)
X, y, categorical_indicator, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
categorical_feature_indices = [idx for idx, idx_bool in enumerate(categorical_indicator) if idx_bool]

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

print("Training set size:", len(X_train))
print("Validation set size:", len(X_valid))
print("Test set size:", len(X_test))

X_train.head()

Training set size: 13209
Validation set size: 3303
Test set size: 4128


,median_income,housing_median_age,total_rooms,total_bedrooms,population,households,latitude,longitude
16490,4.7069,27,1683.0,288.0,873.0,258.0,38.00,-120.97
6090,3.8750,15,6409.0,1363.0,3359.0,1267.0,34.10,-117.87
10973,2.8828,26,2312.0,525.0,1273.0,437.0,33.76,-117.85
17287,5.0000,34,1664.0,292.0,705.0,257.0,34.35,-119.74
12615,5.0371,25,4719.0,745.0,1857.0,739.0,38.50,-121.51


In [4]:
from GRANDE import GRANDE

params = {
        'depth': 5, # tree depth
        'n_estimators': 2048, # number of estimators / trees

        'learning_rate_weights': 0.005, # learning rate for leaf weights
        'learning_rate_index': 0.01, # learning rate for split indices
        'learning_rate_values': 0.01, # learning rate for split values
        'learning_rate_leaf': 0.01, # learning rate for leafs (logits)

        'optimizer': 'adam', # optimizer
        'cosine_decay_steps': 0, # decay steps for lr schedule (CosineDecayRestarts)

        'loss': 'mse', # loss function (default 'crossentropy' for binary & multi-class classification and 'mse' for regression)
        'focal_loss': False, # use focal loss {True, False}
        'temperature': 0.0, # temperature for stochastic re-weighted GD (0.0, 1.0)

        'from_logits': True, # use logits for weighting {True, False}
        'use_class_weights': True, # use class weights for training {True, False}

        'dropout': 0.0, # dropout rate (here, dropout randomly disables individual estimators of the ensemble during training)

        'selected_variables': 0.8, # feature subset percentage (0.0, 1.0)
        'data_subset_fraction': 1.0, # data subset percentage (0.0, 1.0)
}

args = {
    'epochs': 1_000, # number of epochs for training
    'early_stopping_epochs': 25, # patience for early stopping (best weights are restored)
    'batch_size': 64,  # batch size for training

    'cat_idx': categorical_feature_indices, # put list of categorical indices
    'objective': 'regression', # objective / task {'binary', 'classification', 'regression'}
    
    'random_seed': 42,
    'verbose': 1,       
}

model_grande = GRANDE(params=params, args=args)

model_grande.fit(X_train=X_train,
          y_train=y_train,
          X_val=X_valid,
          y_val=y_valid)

preds_grande = model_grande.predict(X_test)

2024-03-15 13:03:31.094292: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-15 13:03:32.579697: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-03-15 13:03:37.579833: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-03-15 13:03:37.580326: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 34057 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:61:00.0, compute capability: 8.6


Epoch 1/1000


I0000 00:00:1710504221.027930 1575662 service.cc:145] XLA service 0x7fc2283984c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1710504221.027977 1575662 service.cc:153]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-03-15 13:03:41.100591: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-15 13:03:41.362455: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902


 58/207 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9063

I0000 00:00:1710504222.078759 1575662 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


207/207 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.7627 - val_loss: 0.4361
Epoch 2/1000
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3708 - val_loss: 0.3173
Epoch 3/1000
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2854 - val_loss: 0.2777
Epoch 4/1000
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2531 - val_loss: 0.2563
Epoch 5/1000
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2356 - val_loss: 0.2443
Epoch 6/1000
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2242 - val_loss: 0.2401
Epoch 7/1000
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2165 - val_loss: 0.2336
Epoch 8/1000
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2100 - val_loss: 0.2279
Epoch 9/1000
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2054 - val_loss: 0.2241
Epoch 10/1000
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2004 - val_loss: 0.2196
Epoch 11/1000
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1969 - val_loss: 0.2180
Epoch 12/1000
207/207 ━━━━━━━━━━━━━━━━━━━━

In [5]:
def calculate_sample_weights(y_data):
    class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight = 'balanced', classes = np.unique(y_data), y = y_data)
    sample_weights = sklearn.utils.class_weight.compute_sample_weight(class_weight = 'balanced', y =y_data)
    return sample_weights

In [6]:
try:
    y_train = y_train.values.codes.astype(np.float64)
    y_valid = y_valid.values.codes.astype(np.float64)
    y_test = y_test.values.codes.astype(np.float64)
except:
    y_train = y_train.values.astype(np.float64)
    y_valid = y_valid.values.astype(np.float64)
    y_test = y_test.values.astype(np.float64)

In [7]:
binary_indices = []
low_cardinality_indices = []
high_cardinality_indices = []
num_columns = []
for column_index, column in enumerate(X_train.columns):
    if column_index in categorical_feature_indices:
        if len(X_train.iloc[:,column_index].unique()) <= 2:
            binary_indices.append(column)
        if len(X_train.iloc[:,column_index].unique()) < 5:
            low_cardinality_indices.append(column)
        else:
            high_cardinality_indices.append(column)
    else:
        num_columns.append(column)     
cat_columns = [col for col in X_train.columns if col not in num_columns]

In [8]:
if len(num_columns) > 0:
    mean_train_num = X_train[num_columns].mean(axis=0).iloc[0]
    X_train[num_columns] = X_train[num_columns].fillna(mean_train_num)
    X_valid[num_columns] = X_valid[num_columns].fillna(mean_train_num)
    X_test[num_columns] = X_test[num_columns].fillna(mean_train_num)
if len(cat_columns) > 0:
    mode_train_cat = X_train[cat_columns].mode(axis=0).iloc[0]
    X_train[cat_columns] = X_train[cat_columns].fillna(mode_train_cat)
    X_valid[cat_columns] = X_valid[cat_columns].fillna(mode_train_cat)
    X_test[cat_columns] = X_test[cat_columns].fillna(mode_train_cat)

X_train_raw = X_train.copy()
X_valid_raw = X_valid.copy()
X_test_raw = X_test.copy()

In [9]:
encoder_ordinal = ce.OrdinalEncoder(cols=binary_indices)
encoder_ordinal.fit(X_train)
X_train = encoder_ordinal.transform(X_train)
X_valid = encoder_ordinal.transform(X_valid)  
X_test = encoder_ordinal.transform(X_test)       

encoder = ce.LeaveOneOutEncoder(cols=high_cardinality_indices)
encoder.fit(X_train, y_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

encoder = ce.OneHotEncoder(cols=low_cardinality_indices)
encoder.fit(X_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

X_train = X_train.astype(np.float32)
X_valid = X_valid.astype(np.float32)
X_test = X_test.astype(np.float32)

In [10]:
from xgboost import XGBRegressor
model_xgb = XGBRegressor(n_estimators=1000, early_stopping_rounds=20)
model_xgb.fit(X_train, 
                y_train, 
                sample_weight=calculate_sample_weights(y_train), 
                eval_set=[(X_valid, y_valid)], 
                sample_weight_eval_set=[calculate_sample_weights(y_valid)]
             )
if args['objective'] == 'regression':
    preds_xgb = model_xgb.predict(X_test)
else:
    preds_xgb = model_xgb.predict_proba(X_test)

[0]	validation_0-rmse:167443.45444
[1]	validation_0-rmse:123459.94133
[2]	validation_0-rmse:95129.67696
[3]	validation_0-rmse:77096.46133
[4]	validation_0-rmse:66935.05863
[5]	validation_0-rmse:61017.07684
[6]	validation_0-rmse:57753.85036
[7]	validation_0-rmse:55921.86456
[8]	validation_0-rmse:53895.20276
[9]	validation_0-rmse:52756.79624
[10]	validation_0-rmse:52009.06597
[11]	validation_0-rmse:51647.01177
[12]	validation_0-rmse:50420.07531
[13]	validation_0-rmse:50068.95575
[14]	validation_0-rmse:49828.91226
[15]	validation_0-rmse:49635.05670
[16]	validation_0-rmse:49093.20613
[17]	validation_0-rmse:48757.50206
[18]	validation_0-rmse:48699.77755
[19]	validation_0-rmse:48637.91139
[20]	validation_0-rmse:48591.48668
[21]	validation_0-rmse:48258.77761
[22]	validation_0-rmse:48143.79682
[23]	validation_0-rmse:48088.35506
[24]	validation_0-rmse:48002.74794
[25]	validation_0-rmse:47880.60813
[26]	validation_0-rmse:47753.95386
[27]	validation_0-rmse:47777.41832
[28]	validation_0-rmse:47702

In [11]:
from catboost import CatBoostRegressor, Pool

model_catboost = CatBoostRegressor(n_estimators=1000, 
                                    early_stopping_rounds=20)
train_data = Pool(
        data=X_train_raw,
        label=y_train,
        cat_features=categorical_feature_indices,
        weight=calculate_sample_weights(y_train)
    )

eval_data = Pool(
        data=X_valid_raw,
        label=y_valid,
        cat_features=categorical_feature_indices,
        weight=calculate_sample_weights(y_valid),
    )

model_catboost.fit(X=train_data, 
                   eval_set=eval_data)

if args['objective'] == 'regression':
    preds_catboost = model_catboost.predict(X_test_raw)
else:
    preds_catboost = model_catboost.predict_proba(X_test_raw)


Learning rate set to 0.076361
0:	learn: 112413.6920108	test: 104827.1150993	best: 104827.1150993 (0)	total: 50.7ms	remaining: 50.7s
1:	learn: 108432.7221618	test: 101135.9559217	best: 101135.9559217 (1)	total: 54.9ms	remaining: 27.4s
2:	learn: 104873.7852911	test: 97747.7089652	best: 97747.7089652 (2)	total: 57.9ms	remaining: 19.3s
3:	learn: 101662.2574261	test: 94696.0484905	best: 94696.0484905 (3)	total: 60.7ms	remaining: 15.1s
4:	learn: 98328.0535611	test: 91562.6624424	best: 91562.6624424 (4)	total: 63.6ms	remaining: 12.7s
5:	learn: 95521.8734728	test: 88943.6284296	best: 88943.6284296 (5)	total: 66.4ms	remaining: 11s
6:	learn: 92760.0099356	test: 86362.9039896	best: 86362.9039896 (6)	total: 69.1ms	remaining: 9.8s
7:	learn: 90270.6254827	test: 84022.9547964	best: 84022.9547964 (7)	total: 71.9ms	remaining: 8.91s
8:	learn: 88352.4862758	test: 82318.7282439	best: 82318.7282439 (8)	total: 74.3ms	remaining: 8.18s
9:	learn: 86182.7273301	test: 80336.0797330	best: 80336.0797330 (9)	total:

In [12]:
if args['objective'] == 'binary':
    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_grande[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_grande[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande[:,1], average='macro', multi_class='ovo')

    print('Accuracy GRANDE:', accuracy)
    print('F1 Score GRANDE:', f1_score)
    print('ROC AUC GRANDE:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_xgb[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_xgb[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb[:,1], average='macro', multi_class='ovo')

    print('Accuracy XGB:', accuracy)
    print('F1 Score XGB:', f1_score)
    print('ROC AUC XGB:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_catboost[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_catboost[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost[:,1], average='macro', multi_class='ovo')

    print('Accuracy CatBoost:', accuracy)
    print('F1 Score CatBoost:', f1_score)
    print('ROC AUC CatBoost:', roc_auc)
    print('\n')
elif args['objective'] == 'classification':
    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_grande, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_grande, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy GRANDE:', accuracy)
    print('F1 Score GRANDE:', f1_score)
    print('ROC AUC GRANDE:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_xgb, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_xgb, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy XGB:', accuracy)
    print('F1 Score XGB:', f1_score)
    print('ROC AUC XGB:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_catboost, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_catboost, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy CatBoost:', accuracy)
    print('F1 Score CatBoost:', f1_score)
    print('ROC AUC CatBoost:', roc_auc)
    print('\n')
else:
    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_grande))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_grande))

    print('MAE GRANDE:', mean_absolute_error)
    print('R2 Score GRANDE:', r2_score)
    print('\n')

    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_xgb))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_xgb))

    print('MAE XGB:', mean_absolute_error)
    print('R2 Score XGB:', r2_score)
    print('\n')

    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_catboost))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_catboost))

    print('MAE CatBoost:', mean_absolute_error)
    print('R2 Score CatBoost:', r2_score)
    print('\n')

MAE GRANDE: 34858.060562015504
R2 Score GRANDE: 0.8039679642680202


MAE XGB: 34096.338662790695
R2 Score XGB: 0.811414696015542


MAE CatBoost: 32114.8941375969
R2 Score CatBoost: 0.8256524499655538




In [13]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()